In [2]:
import random
import pennylane as qml
import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader, TensorDataset, random_split
import torch.nn as nn
from torch.optim import Adam
from tqdm import tqdm
import copy
from sklearn.metrics import roc_auc_score
import copy
import time
from typing import Any, Optional, Tuple, Callable
import mne
from sklearn.model_selection import train_test_split
import math

print('Pennylane Version :', qml.__version__)
print('Pytorch Version :', torch.__version__)

Pennylane Version : 0.41.1
Pytorch Version : 2.7.1+cu126


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = "cpu"
print("Running on ", device)

Running on  cuda


# Prepare PhysioNet EEG Dataset

In [ ]:
def load_eeg_ts(seed, device, batch_size, sampling_freq):
    # Set random seed for reproducibility
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        
    # Load and preprocess the PhysioNet EEG Motor Imagery data
    N_SUBJECT = 50
    IMAGINE_OPEN_CLOSE_LEFT_RIGHT_FIST = [4, 8, 12]

    # Load data from PhysioNet (example assumes data is downloaded locally)
    physionet_paths = [
        mne.datasets.eegbci.load_data(
            subjects=subj_id,
            runs=IMAGINE_OPEN_CLOSE_LEFT_RIGHT_FIST,
            path="PhysioNet_EEG",
        ) for subj_id in range(1, N_SUBJECT+1)
    ]
    physionet_paths = np.concatenate(physionet_paths)

    # Ensuring that all subjects share same sampling frequency
    # TARGET_SFREQ = 160  # 160 Hz sampling rate
    TARGET_SFREQ = sampling_freq

    # Concatenate all loaded raw data
    parts = []
    for path in physionet_paths:
        raw = mne.io.read_raw_edf(
            path,
            preload=True,
            stim_channel='auto',
            verbose='WARNING',
        )
        # Resample raw data to ensure consistent sfreq
        raw.resample(TARGET_SFREQ, npad="auto")
        parts.append(raw)
        
    # Concatenate resampled raw data
    raw = mne.concatenate_raws(parts)

    # Pick EEG channels and extract events
    eeg_channel_inds = mne.pick_types(
        raw.info, meg=False, eeg=True, stim=False, eog=False, exclude='bads'
    )
    events, _ = mne.events_from_annotations(raw)

    # Epoch the data
    epoched = mne.Epochs(
        raw, events, dict(left=2, right=3), tmin=1, tmax=4.1,
        proj=False, picks=eeg_channel_inds, baseline=None, preload=True
    )

    # Convert data to NumPy arrays
    X = (epoched.get_data() * 1e3).astype(np.float32)  # Convert to millivolts
    y = (epoched.events[:, 2] - 2).astype(np.int64)  # 0: left, 1: right
    
    # Train-validation-test split
    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=seed)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=seed)
    
    def MakeTensorDataset(X, y):
        X_tensor = torch.tensor(X, dtype=torch.float32).to(device)
        y_tensor = torch.tensor(y, dtype=torch.float32).to(device)
        tensordataset = TensorDataset(X_tensor, y_tensor)
        return tensordataset
    
    # Create datasets and dataloaders
    train_dataset = MakeTensorDataset(X_train, y_train)
    val_dataset = MakeTensorDataset(X_val, y_val)
    test_dataset = MakeTensorDataset(X_test, y_test)

    # DataLoaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    
    input_dim = X_train.shape
    
    return train_loader, val_loader, test_loader, input_dim

In [5]:
import numpy as np
import torch
from torch.utils.data import DataLoader, TensorDataset
import mne
from sklearn.model_selection import train_test_split

print('MNE Version :', mne.__version__)

def load_eeg_ts_revised(seed, device, batch_size, sampling_freq, sample_size):
    """
    Loads and preprocesses the PhysioNet EEG Motor Imagery dataset for a specified number of subjects.

    Args:
        seed (int): Random seed for reproducibility.
        device (torch.device): The device to move the tensors to.
        batch_size (int): Number of samples per batch.
        sampling_freq (int): The target sampling frequency to resample the data to.
        sample_size (int): The number of subjects to load data from (1 to 109).

    Returns:
        tuple: A tuple containing (train_loader, test_loader, input_dim).
               - train_loader: DataLoader for the training set.
               - val_loader: DataLoader for the validation set.
               - test_loader: DataLoader for the test set.
               - input_dim: The shape of the input data (n_trials, n_channels, n_timesteps).
    """
    
    # --- Step 1: Split Subject IDs into Train, Validation, and Test Sets ---
    N_SUBJECT = sample_size
    subject_ids = np.arange(1, N_SUBJECT + 1)
    
    # Split subjects: ~70% train, ~15% validation, ~15% test
    train_subjects, temp_subjects = train_test_split(
        subject_ids,
        test_size=0.3, # 30% for validation and test combined
        random_state=seed
    )
    
    val_subjects, test_subjects = train_test_split(
        temp_subjects,
        test_size=0.5, # Split the 30% evenly into 15% validation and 15% test
        random_state=seed
    )
    
    print(f"Subjects in Training Set: {len(train_subjects)}")
    print(f"Subjects in Validation Set: {len(val_subjects)}")
    print(f"Subjects in Test Set: {len(test_subjects)}")


    # ⬇️ 추가: 보기 좋게 정렬 + 겹침 여부 검증 + 화면/파일 출력
    train_subjects_sort = np.sort(train_subjects)
    val_subjects_sort   = np.sort(val_subjects)
    test_subjects_sort  = np.sort(test_subjects)

    # 겹치는지 안전 체크
    assert not set(train_subjects_sort) & set(val_subjects_sort)
    assert not set(train_subjects_sort) & set(test_subjects_sort)
    assert not set(val_subjects_sort)   & set(test_subjects_sort)

    print(f"Train subjects ({len(train_subjects_sort)}): {train_subjects_sort.tolist()}")
    print(f"Val subjects   ({len(val_subjects_sort)}): {val_subjects_sort.tolist()}")
    print(f"Test subjects  ({len(test_subjects_sort)}): {test_subjects_sort.tolist()}")

    # 필요하면 텍스트 파일로도 남기기 (옵션)
    np.savetxt(f"results/tcn{seed}_{sample_size}samples_train_subjects.txt", train_subjects_sort, fmt="%d")
    np.savetxt(f"results/tcn{seed}_{sample_size}samples_val_subjects.txt",   val_subjects_sort,   fmt="%d")
    np.savetxt(f"results/tcn{seed}_{sample_size}samples_test_subjects.txt",  test_subjects_sort,  fmt="%d")
    

    # --- Step 2: Define a Helper Function to Load Data for a Given List of Subjects ---
    def _load_and_process_subjects(subject_list, sfreq):
        IMAGINE_OPEN_CLOSE_LEFT_RIGHT_FIST = [4, 8, 12]
        
        # Load file paths for the specified subjects
        physionet_paths = [
            mne.datasets.eegbci.load_data(
                subjects=subj_id,
                runs=IMAGINE_OPEN_CLOSE_LEFT_RIGHT_FIST,
                path="PhysioNet_EEG",
            ) for subj_id in subject_list
        ]
        physionet_paths = np.concatenate(physionet_paths)

        parts = []
        for path in physionet_paths:
            raw = mne.io.read_raw_edf(
                path, preload=True, stim_channel='auto', verbose='WARNING'
            )
            raw.resample(sfreq, npad="auto")
            parts.append(raw)
        
        # Concatenate all runs FOR THIS SPLIT into one raw object
        raw = mne.concatenate_raws(parts)

        # Epoch the data
        events, _ = mne.events_from_annotations(raw)
        eeg_channel_inds = mne.pick_types(
            raw.info, meg=False, eeg=True, stim=False, eog=False, exclude='bads'
        )
        epoched = mne.Epochs(
            raw, events, dict(left=2, right=3), tmin=1, tmax=4.1,
            proj=False, picks=eeg_channel_inds, baseline=None, preload=True
        )

        # Convert to NumPy arrays
        X = (epoched.get_data() * 1e3).astype(np.float32)
        y = (epoched.events[:, 2] - 2).astype(np.int64)
        
        return X, y

    # --- Step 3: Load Data Separately for Each Subject Group ---
    X_train, y_train = _load_and_process_subjects(train_subjects, sampling_freq)
    X_val, y_val = _load_and_process_subjects(val_subjects, sampling_freq)
    X_test, y_test = _load_and_process_subjects(test_subjects, sampling_freq)
    
    print(f"\nTraining set shape: {X_train.shape}")
    print(f"Validation set shape: {X_val.shape}")
    print(f"Test set shape: {X_test.shape}")

    # --- Step 4: Create PyTorch Datasets and DataLoaders ---
    train_dataset = TensorDataset(torch.tensor(X_train, dtype=torch.float32).to(device),
                                  torch.tensor(y_train, dtype=torch.float32).to(device))
    val_dataset = TensorDataset(torch.tensor(X_val, dtype=torch.float32).to(device),
                                torch.tensor(y_val, dtype=torch.float32).to(device))
    test_dataset = TensorDataset(torch.tensor(X_test, dtype=torch.float32).to(device),
                                 torch.tensor(y_test, dtype=torch.float32).to(device))

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    
    input_dim = X_train.shape
    
    return train_loader, val_loader, test_loader, input_dim

MNE Version : 1.9.0


In [ ]:
SEED = 2025
SAMPLE_SIZE = 10

train_loader, val_loader, test_loader, input_dim = load_eeg_ts_revised(seed=SEED, device=device, batch_size=32, sampling_freq=4, sample_size=SAMPLE_SIZE)

Subjects in Training Set: 7
Subjects in Validation Set: 1
Subjects in Test Set: 2
Train subjects (7): [1, 4, 6, 7, 8, 9, 10]
Val subjects   (1): [3]
Test subjects  (2): [2, 5]
Used Annotations descriptions: [np.str_('T0'), np.str_('T1'), np.str_('T2')]
Not setting metadata
315 matching events found
No baseline correction applied
Using data from preloaded Raw for 315 events and 13 original time points ...
15 bad epochs dropped
Used Annotations descriptions: [np.str_('T0'), np.str_('T1'), np.str_('T2')]
Not setting metadata
45 matching events found
No baseline correction applied
Using data from preloaded Raw for 45 events and 13 original time points ...
0 bad epochs dropped
Used Annotations descriptions: [np.str_('T0'), np.str_('T1'), np.str_('T2')]
Not setting metadata
90 matching events found
No baseline correction applied
Using data from preloaded Raw for 90 events and 13 original time points ...
6 bad epochs dropped

Training set shape: (300, 64, 13)
Validation set shape: (45, 64, 13

In [7]:
next(iter(test_loader))

[tensor([[[ 0.0143,  0.1387, -0.1181,  ..., -0.0439, -0.0123, -0.0770],
          [-0.0423,  0.1146, -0.1215,  ...,  0.0015,  0.0134, -0.0489],
          [-0.0365,  0.1278, -0.1096,  ...,  0.0033,  0.0209, -0.0387],
          ...,
          [-0.0422,  0.0359, -0.0298,  ..., -0.0088,  0.0111, -0.0122],
          [-0.0908,  0.0568,  0.0224,  ..., -0.0119,  0.0121, -0.0338],
          [-0.0653, -0.0059, -0.0178,  ...,  0.0064, -0.0029, -0.0376]],
 
         [[-0.0480,  0.1008, -0.0787,  ...,  0.0236,  0.0382,  0.1542],
          [-0.0689,  0.0874, -0.1231,  ...,  0.0074,  0.0280,  0.1289],
          [-0.0607,  0.1121, -0.1092,  ...,  0.0666,  0.0698,  0.1549],
          ...,
          [ 0.0032,  0.0220, -0.0426,  ...,  0.0947,  0.1242,  0.1084],
          [ 0.0081,  0.0227, -0.0286,  ...,  0.0756,  0.0871,  0.0657],
          [-0.0154, -0.0076, -0.0498,  ...,  0.0600,  0.0907,  0.0680]],
 
         [[-0.0475, -0.0014,  0.0400,  ..., -0.0095,  0.1064,  0.0189],
          [-0.0538, -0.0128,

In [8]:
input_dim

(1492, 64, 13)

In [8]:
from tcn import TemporalConvNet
import torch.nn.functional as F

class EEGTCNClassifier(nn.Module):
    def __init__(self, in_ch, tcn_ch, num_classes, k=2, dropout=0.2):
        super().__init__()
        self.tcn = TemporalConvNet(in_ch, tcn_ch, k, dropout)
        self.linear = nn.Linear(tcn_ch[-1], num_classes)
    def forward(self, x):
        # x: (B, C, L)
        y = self.tcn(x)            # → (B, C_out, L)
        logit = self.linear(y[:, :, -1])
        return logit

In [ ]:
################################# Calculate Running Time ########################################
def epoch_time(start_time: float, end_time: float) -> Tuple[float, float]:
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs


################################# Performance & Density Matrices ################################
# Training loop
def train_perf(model, dataloader, optimizer, criterion):
    model.train()
    train_loss = 0.0
    all_labels = []
    all_outputs = []
    for inputs, labels in tqdm(dataloader):
        assert not torch.isnan(inputs).any(), "Inputs contain NaN!"
        assert not torch.isinf(inputs).any(), "Inputs contain Inf!"
        assert not torch.isnan(labels).any(), "Labels contain NaN!"
        inputs, labels = inputs.to(device), labels.to(device)  # Ensure that data is on the same device (GPU or CPU)
        labels = labels.float()   # Ensure labels are of type float for BCEWithLogitsLoss
        optimizer.zero_grad()
        outputs = model(inputs)
        outputs = outputs.squeeze(1)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        
        # Collect labels and outputs for AUROC
        all_labels.append(labels.cpu().numpy())
        all_outputs.append(outputs.detach().cpu().numpy())       
    # Calculate train AUROC
    all_labels = np.concatenate(all_labels)
    all_outputs = np.concatenate(all_outputs)
    print("labels NaNs:", np.isnan(all_labels).sum(), "/", all_labels.shape[0])
    print("outputs NaNs:", np.isnan(all_outputs).sum(), "/", all_outputs.shape[0])
    train_auroc = roc_auc_score(all_labels, all_outputs)
    
    return train_loss / len(dataloader), train_auroc


# Validation/Test loop
def evaluate_perf(model, dataloader, criterion):
    model.eval()
    running_loss = 0.0
    all_labels = []
    all_outputs = []
    with torch.no_grad():
        for inputs, labels in tqdm(dataloader):
            inputs, labels = inputs.to(device), labels.to(device)  # Ensure that data is on the same device (GPU or CPU)
            labels = labels.float()   # Ensure labels are of type float for BCEWithLogitsLoss
            outputs = model(inputs)
            outputs = outputs.squeeze(1)
            loss = criterion(outputs, labels)
            running_loss += loss.item()

            # Collect labels and outputs for AUROC
            all_labels.append(labels.cpu().numpy())
            all_outputs.append(outputs.cpu().numpy())

    all_labels = np.concatenate(all_labels)
    all_outputs = np.concatenate(all_outputs)
    auroc = roc_auc_score(all_labels, all_outputs)
    
    return running_loss / len(dataloader), auroc

In [33]:
SEED = 2027
SAMPLE_SIZE = 50

train_loader, val_loader, test_loader, input_dim = load_eeg_ts_revised(seed=SEED, device=device, batch_size=32, sampling_freq=4, sample_size=SAMPLE_SIZE)

Subjects in Training Set: 35
Subjects in Validation Set: 7
Subjects in Test Set: 8
Train subjects (35): [1, 3, 4, 5, 8, 9, 10, 12, 13, 14, 15, 17, 18, 20, 23, 25, 27, 28, 29, 30, 31, 32, 33, 34, 36, 37, 38, 39, 41, 43, 44, 45, 46, 47, 48]
Val subjects   (7): [6, 11, 24, 26, 35, 42, 49]
Test subjects  (8): [2, 7, 16, 19, 21, 22, 40, 50]
Used Annotations descriptions: [np.str_('T0'), np.str_('T1'), np.str_('T2')]
Not setting metadata
1575 matching events found
No baseline correction applied
Using data from preloaded Raw for 1575 events and 13 original time points ...
83 bad epochs dropped
Used Annotations descriptions: [np.str_('T0'), np.str_('T1'), np.str_('T2')]
Not setting metadata
315 matching events found
No baseline correction applied
Using data from preloaded Raw for 315 events and 13 original time points ...
18 bad epochs dropped
Used Annotations descriptions: [np.str_('T0'), np.str_('T1'), np.str_('T2')]
Not setting metadata
360 matching events found
No baseline correction appli

In [ ]:
num_epochs = 20  # Set the number of epochs for training

model = EEGTCNClassifier(in_ch=64, tcn_ch=[16, 16, 16], num_classes=1, k=2, dropout=0.3).to(device)
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Total: {total_params:,}  |  Trainable: {trainable_params:,}  |  Frozen: {total_params-trainable_params:,}")

lr = 0.0005
criterion = nn.BCEWithLogitsLoss()  # Use BCEWithLogitsLoss for binary classification
optimizer = Adam(model.parameters(), lr=lr, weight_decay=1e-4, eps=1e-8)

# Training process
train_metrics, valid_metrics, test_metrics = [], [], []

for epoch in range(num_epochs):
    start_time = time.time()
    
    train_loss, train_auc = train_perf(model, train_loader, optimizer, criterion)
    train_metrics.append({'epoch': epoch + 1, 'train_loss': train_loss, 'train_auc': train_auc})    

    valid_loss, valid_auc = evaluate_perf(model, val_loader, criterion)
    valid_metrics.append({'epoch': epoch + 1, 'valid_loss': valid_loss, 'valid_auc': valid_auc})

    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    print(f"Epoch: {epoch + 1:02} | Time: {epoch_mins}m {epoch_secs}s")
    print(f"Train Loss: {train_loss:.4f}, AUC: {train_auc:.4f} | Validation Loss: {valid_loss:.4f}, AUC: {valid_auc:.4f}")

# Final evaluation on the test set
test_loss, test_auc = evaluate_perf(model, test_loader, criterion)
print(f"Test Loss: {test_loss:.4f}, AUC: {test_auc:.4f}")
test_metrics.append({'epoch': num_epochs, 'test_loss': test_loss, 'test_auc': test_auc}) 

# Combine all metrics into a pandas DataFrame
metrics = []
for epoch in range(num_epochs):
    metrics.append({
        'epoch': epoch + 1,
        'train_loss': train_metrics[epoch]['train_loss'],
        'train_auc': train_metrics[epoch]['train_auc'],
        'valid_loss': valid_metrics[epoch]['valid_loss'],
        'valid_auc': valid_metrics[epoch]['valid_auc'],
        'test_loss': test_metrics[0]['test_loss'],
        'test_auc': test_metrics[0]['test_auc'],
    })
# Convert to DataFrame
metrics_df = pd.DataFrame(metrics)
# Save to CSV
csv_filename = f"results/TCN_performance_{SEED}_{SAMPLE_SIZE}samples_lr{lr}_small.csv"
metrics_df.to_csv(csv_filename, index=False)
print(f"Metrics saved to {csv_filename}")

/global/homes/a/alveare/.conda/envs/qml/lib/python3.11/site-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


Total: 5,857  |  Trainable: 5,857  |  Frozen: 0


  0%|          | 0/47 [00:00<?, ?it/s]

100%|██████████| 47/47 [00:00<00:00, 89.09it/s]


labels NaNs: 0 / 1492
outputs NaNs: 0 / 1492


...

labels NaNs: 0 / 1492
outputs NaNs: 0 / 1492


100%|██████████| 10/10 [00:00<00:00, 380.29it/s]


Epoch: 20 | Time: 0m 0s
Train Loss: 0.5847, AUC: 0.7791 | Validation Loss: 0.6188, AUC: 0.7505


100%|██████████| 11/11 [00:00<00:00, 368.63it/s]

Test Loss: 0.5799, AUC: 0.8026
Metrics saved to results/TCN_performance_2027_50samples_lr0.0005_small.csv
